In [201]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

In [202]:
file_path = r"E:\project\youtube\data\ZB_ML_성시경&최자로드.xlsx"

sung_df = pd.read_excel(file_path, sheet_name=1)

choi_df1 = pd.read_excel(file_path, sheet_name=3)
choi_df2 = pd.read_excel(file_path, sheet_name=2)
choi_df = pd.concat([choi_df1, choi_df2], axis=0, ignore_index=True)

In [203]:
type(choi_df['guest_name'][2])

str

In [204]:
sung_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   content_name           151 non-null    object 
 1   restaurant_name        151 non-null    object 
 2   restaurant_region      151 non-null    object 
 3   food_category          150 non-null    object 
 4   guest_num              149 non-null    float64
 5   guest_name             28 non-null     object 
 6   most_moment_time_1     151 non-null    object 
 7   most_moment_comment_1  151 non-null    object 
 8   most_moment_time_2     85 non-null     object 
 9   most_moment_comment_2  85 non-null     object 
 10  most_moment_time_3     36 non-null     object 
 11  most_moment_comment_3  36 non-null     object 
 12  Unnamed: 12            7 non-null      object 
 13  Unnamed: 13            7 non-null      object 
 14  Unnamed: 14            4 non-null      object 
 15  Unname

In [205]:
choi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   content_name           108 non-null    object 
 1   restaurant_name        107 non-null    object 
 2   restaurant_region      106 non-null    object 
 3   food_category          106 non-null    object 
 4   guest_num              108 non-null    int64  
 5   guest_name             68 non-null     object 
 6   most_moment_time_1     107 non-null    object 
 7   most_moment_comment_1  107 non-null    object 
 8   most_moment_time_2     48 non-null     object 
 9   most_moment_comment_2  48 non-null     object 
 10  most_moment_time_3     19 non-null     object 
 11  most_moment_comment_3  19 non-null     object 
 12  Unnamed: 12            10 non-null     object 
 13  Unnamed: 13            14 non-null     object 
 14  Unnamed: 14            4 non-null      object 
 15  Unname

# 게스트 이름 리스트

In [206]:
def guest_name_list(info_df):
    guest_series_list = info_df['guest_name']
    name_list_notnull = guest_series_list[guest_series_list.notnull()]
    sorted_name_list = pd.Series.reset_index(name_list_notnull, drop=True)
    name_list = []
    for name in sorted_name_list:
        name = name.upper()
        if ',' in name:
            splited_name = name.split(',')
            for i in splited_name:
                if i.strip() not in name_list:
                    name_list.append(i.strip())
        else:
            if name not in name_list:
                name_list.append(name.strip())
    return name_list

In [207]:
sung_guest_list = guest_name_list(sung_df)
len(sung_guest_list), sung_guest_list

(18,
 ['유세윤',
  '문천식',
  '양희은',
  '백종원',
  '신동엽',
  '마츠다상',
  '구독자',
  '다나카',
  '싸이',
  '하정우',
  '주지훈',
  '성시경 매형(마틴)',
  '최자',
  '에픽하이',
  '박준우(쉐프)',
  '크리스탈 케이',
  '초아',
  '케이윌'])

In [208]:
choi_guest_list = guest_name_list(choi_df)
len(choi_guest_list), choi_guest_list

(66,
 ['CHOON6',
  'MANGO FRESH',
  'DJ PANDA',
  '맹진환',
  '개코',
  '던밀스',
  '한해',
  '유세윤',
  '뮤지',
  '붐',
  'WOOGIE',
  'DJ FRIZ',
  '페노메코',
  'APRO',
  '행주',
  '이건우(최자로드PD)',
  '성시경',
  '찬열',
  '세훈',
  '개코 이기백',
  '황윤창',
  '주범수',
  '밥굽남',
  '김수미',
  '한창헌PD',
  '와디의 신발장',
  '가오가이',
  '권민택',
  '미쓰라',
  'THAMA',
  '이종현',
  '핫펠트',
  '선재',
  '브루노 챔프맨',
  '허성현',
  '언텔',
  '보이비',
  '차메인',
  '와디',
  '빅디',
  '설채현',
  '핫산',
  '릴러말즈',
  'DH FRIZ',
  '피식대학(민수',
  '용주',
  '재형)',
  '숏박스(원훈',
  '진세)',
  '모니카',
  '지코',
  '존박',
  '안재홍',
  '맛타고라스',
  'PH-1',
  'JUNNY',
  '비와이',
  '김재환',
  'PADI',
  '김풍',
  '육식맨',
  '양동근',
  'SOLE',
  '엄지윤',
  '이성우',
  '박종윤'])

# 게스트 정보
- 이름
- 직업
- 나이
- 고향

In [209]:
sung_df['guest_num'].max(), choi_df['guest_num'].max()

(3.0, 3)

In [210]:
sung_df['guest_name1'] = np.nan
sung_df['guest_job1'] = np.nan
sung_df['guest_age1'] = np.nan
sung_df['guest_hometown1'] = np.nan
sung_df['guest_name2'] = np.nan
sung_df['guest_job2'] = np.nan
sung_df['guest_age2'] = np.nan
sung_df['guest_hometown2'] = np.nan
sung_df['guest_name3'] = np.nan
sung_df['guest_job3'] = np.nan
sung_df['guest_age3'] = np.nan
sung_df['guest_hometown3'] = np.nan

In [211]:
choi_df['guest_name1'] = np.nan
choi_df['guest_job1'] = np.nan
choi_df['guest_age1'] = np.nan
choi_df['guest_hometown1'] = np.nan
choi_df['guest_name2'] = np.nan
choi_df['guest_job2'] = np.nan
choi_df['guest_age2'] = np.nan
choi_df['guest_hometown2'] = np.nan
choi_df['guest_name3'] = np.nan
choi_df['guest_job3'] = np.nan
choi_df['guest_age3'] = np.nan
choi_df['guest_hometown3'] = np.nan

In [212]:
choi_df.columns

Index(['content_name', 'restaurant_name', 'restaurant_region', 'food_category',
       'guest_num', 'guest_name', 'most_moment_time_1',
       'most_moment_comment_1', 'most_moment_time_2', 'most_moment_comment_2',
       'most_moment_time_3', 'most_moment_comment_3', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'restaurant_region_raw',
       'guest_name1', 'guest_job1', 'guest_age1', 'guest_hometown1',
       'guest_name2', 'guest_job2', 'guest_age2', 'guest_hometown2',
       'guest_name3', 'guest_job3', 'guest_age3', 'guest_hometown3'],
      dtype='object')

In [213]:
def wikipedia_celeb_info(celebrity_name):
    search_url = f"https://ko.wikipedia.org/wiki/{celebrity_name}"
    response = requests.get(search_url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        info_box = soup.find("table", {"class": "infobox"})
        if info_box:
            rows = info_box.find_all("tr")
            celeb_info = {}
            for row in rows:
                th = row.find("th")
                td = row.find("td")
                if th and td:
                    key = th.text.strip()
                    value = td.text.strip()
                    celeb_info[key] = value
            
            # 직업과 나이 추출
            job = celeb_info.get("직업", "정보 없음")
            birth = celeb_info.get("출생", "정보 없음")
            if birth != "정보 없음":
                try:
                    age = re.search(r'(\d+)세', birth).group(1)
                except:
                    age = "정보 없음"
                try:
                    hometown = re.search(r'\(\d+세\)(.*)', birth).group(1)
                except:
                    hometown = "정보 없음"
            else:
                age = "정보 없음"
                hometown= "정보 없음"
            
            return {
                "name": celebrity_name,
                "job": job,
                "age": age,
                "hometown": hometown,
            }
        else:
            return {"name": celebrity_name, "정보": "인포박스가 없습니다."}
    else:
        return {"name": celebrity_name, "정보": "연결 실패"}
        

In [214]:
sung_info = []
for celeb in sung_guest_list:
    sung_info.append(wikipedia_celeb_info(celeb))

sung_info

[{'name': '유세윤',
  'job': '정보 없음',
  'age': '44',
  'hometown': '대한민국 서울특별시 노원구 상계동[1]'},
 {'name': '문천식',
  'job': '배우, 쇼호스트',
  'age': '47',
  'hometown': '경기도 화성군 반월면(경기도 안산시)'},
 {'name': '양희은',
  'job': '가수  작사가  방송인  배우',
  'age': '72',
  'hometown': '대한민국 서울특별시 종로구 가회동'},
 {'name': '백종원',
  'job': '기업인(더본코리아 대표), 요리연구가, 방송인, 학교법인 이사장(예덕학원)',
  'age': '58',
  'hometown': '대한민국 충청남도 예산군 오가면 역탑리'},
 {'name': '신동엽', '정보': '인포박스가 없습니다.'},
 {'name': '마츠다상', '정보': '연결 실패'},
 {'name': '구독자', '정보': '연결 실패'},
 {'name': '다나카', '정보': '인포박스가 없습니다.'},
 {'name': '싸이',
  'job': '가수, 래퍼, 작사가, 작곡가, 프로듀서',
  'age': '46',
  'hometown': '대한민국 서울특별시 성동구 사근동'},
 {'name': '하정우',
  'job': '배우, 영화감독, 각본가, 영화 제작자, 화가',
  'age': '46',
  'hometown': '대한민국 서울특별시'},
 {'name': '주지훈', 'job': '배우, 모델', 'age': '42', 'hometown': '대한민국 서울특별시'},
 {'name': '성시경 매형(마틴)', '정보': '연결 실패'},
 {'name': '최자',
  'job': '래퍼, 작사가, 작곡가, 프로듀서',
  'age': '44',
  'hometown': '대한민국 서울특별시 강남구 신사동'},
 {'name': '에픽하이', 'job': '정보 없음', 

In [215]:
choi_info = []
for celeb in choi_guest_list:
    choi_info.append(wikipedia_celeb_info(celeb))

choi_info    

[{'name': 'CHOON6', '정보': '연결 실패'},
 {'name': 'MANGO FRESH', '정보': '연결 실패'},
 {'name': 'DJ PANDA', '정보': '연결 실패'},
 {'name': '맹진환', '정보': '연결 실패'},
 {'name': '개코',
  'job': '래퍼, 작사가, 작곡가, 프로듀서',
  'age': '43',
  'hometown': '대한민국 서울특별시 강남구 신사동'},
 {'name': '던밀스', 'job': '래퍼', 'age': '36', 'hometown': ' 대한민국 경기도 수원시'},
 {'name': '한해',
  'job': '래퍼. 작사가, 작곡가, 프로듀서',
  'age': '34',
  'hometown': '대한민국 부산광역시'},
 {'name': '유세윤',
  'job': '정보 없음',
  'age': '44',
  'hometown': '대한민국 서울특별시 노원구 상계동[1]'},
 {'name': '뮤지', '정보': '인포박스가 없습니다.'},
 {'name': '붐', '정보': '인포박스가 없습니다.'},
 {'name': 'WOOGIE', '정보': '연결 실패'},
 {'name': 'DJ FRIZ', '정보': '연결 실패'},
 {'name': '페노메코', 'job': '아티스트, 래퍼', 'age': '31', 'hometown': '대한민국'},
 {'name': 'APRO', '정보': '연결 실패'},
 {'name': '행주', '정보': '인포박스가 없습니다.'},
 {'name': '이건우(최자로드PD)', '정보': '연결 실패'},
 {'name': '성시경',
  'job': '가수  작곡가',
  'age': '45',
  'hometown': '대한민국 서울특별시 서초구 반포동'},
 {'name': '찬열',
  'job': '래퍼, 가수, 배우',
  'age': '31',
  'hometown': '대한민국 서울특별

In [216]:
sung_info[0]

{'name': '유세윤',
 'job': '정보 없음',
 'age': '44',
 'hometown': '대한민국 서울특별시 노원구 상계동[1]'}

In [217]:
sung_info[4]

{'name': '신동엽', '정보': '인포박스가 없습니다.'}

In [218]:
sung_info[5]

{'name': '마츠다상', '정보': '연결 실패'}

In [219]:
# 이름이 sung_guest_list와 일치하는 정보 추출 함수
def find_guest_info(guest_name, sung_info):
    for info in sung_info:
        if info.get('name') == guest_name:
            return info
    return None

In [230]:
def update_guest_info(df, info):
    """
    데이터프레임에 guest 정보를 채워 넣는 함수.
    
    Parameters:
    df (pd.DataFrame): 게스트 정보를 추가할 데이터프레임
    info (list): 연예인 정보 리스트

    Returns:
    pd.DataFrame: guest 정보가 추가된 데이터프레임
    """
    # 데이터프레임에 정보를 채워넣기
    for i, row in df.iterrows():
        # guest_name이 NaN이 아닌 경우만 처리
        if isinstance(row['guest_name'], str):
            # 쉼표가 있는 경우 이름을 나눔
            guests = [guest.strip() for guest in row['guest_name'].split(',')] if ',' in row['guest_name'] else [row['guest_name'].strip()]
            

            # 첫 번째 게스트
            if len(guests) > 0:
                guest1_info = find_guest_info(guests[0].strip(), info)
                if guest1_info:
                    df.at[i, 'guest_name1'] = guest1_info.get('name')
                    df.at[i, 'guest_job1'] = guest1_info.get('job')
                    df.at[i, 'guest_age1'] = guest1_info.get('age')
                    df.at[i, 'guest_hometown1'] = guest1_info.get('hometown')
            
            # 두 번째 게스트
            if len(guests) > 1:
                guest2_info = find_guest_info(guests[1].strip(), info)
                if guest2_info:
                    df.at[i, 'guest_name2'] = guest2_info.get('name')
                    df.at[i, 'guest_job2'] = guest2_info.get('job')
                    df.at[i, 'guest_age2'] = guest2_info.get('age')
                    df.at[i, 'guest_hometown2'] = guest2_info.get('hometown')
            
            # 세 번째 게스트
            if len(guests) > 2:
                guest3_info = find_guest_info(guests[2].strip(), info)
                if guest3_info:
                    df.at[i, 'guest_name3'] = guest3_info.get('name')
                    df.at[i, 'guest_job3'] = guest3_info.get('job')
                    df.at[i, 'guest_age3'] = guest3_info.get('age')
                    df.at[i, 'guest_hometown3'] = guest3_info.get('hometown')
    
    return df


In [232]:
# guest 정보를 가진 데이터프레임에 연예인 정보 추가
sung_updated_df = update_guest_info(sung_df, sung_info)
choi_updated_df = update_guest_info(choi_df, choi_info)

In [239]:
sung_updated_df.columns

Index(['content_name', 'restaurant_name', 'restaurant_region', 'food_category',
       'guest_num', 'guest_name', 'most_moment_time_1',
       'most_moment_comment_1', 'most_moment_time_2', 'most_moment_comment_2',
       'most_moment_time_3', 'most_moment_comment_3', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'guest_name1',
       'guest_job1', 'guest_age1', 'guest_hometown1', 'guest_name2',
       'guest_job2', 'guest_age2', 'guest_hometown2', 'guest_name3',
       'guest_job3', 'guest_age3', 'guest_hometown3'],
      dtype='object')

In [240]:
sung_updated_df[sung_updated_df['guest_name'].notnull()][[
    'guest_name', 'guest_name1','guest_job1', 'guest_age1', 'guest_hometown1', 
    'guest_name2','guest_job2', 'guest_age2', 'guest_hometown2', 'guest_name3',
    'guest_job3', 'guest_age3', 'guest_hometown3']]

,guest_name,guest_name1,guest_job1,guest_age1,guest_hometown1,guest_name2,guest_job2,guest_age2,guest_hometown2,guest_name3,guest_job3,guest_age3,guest_hometown3
7,유세윤,유세윤,정보 없음,44,대한민국 서울특별시 노원구 상계동[1],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,문천식,문천식,"배우, 쇼호스트",47,경기도 화성군 반월면(경기도 안산시),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,양희은,양희은,가수 작사가 방송인 배우,72,대한민국 서울특별시 종로구 가회동,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,백종원,백종원,"기업인(더본코리아 대표), 요리연구가, 방송인, 학교법인 이사장(예덕학원)",58,대한민국 충청남도 예산군 오가면 역탑리,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,백종원,백종원,"기업인(더본코리아 대표), 요리연구가, 방송인, 학교법인 이사장(예덕학원)",58,대한민국 충청남도 예산군 오가면 역탑리,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,신동엽,신동엽,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,신동엽,신동엽,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,신동엽,신동엽,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55,마츠다상,마츠다상,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56,마츠다상,마츠다상,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [241]:
choi_updated_df[choi_updated_df['guest_name'].notnull()][[
    'guest_name', 'guest_name1','guest_job1', 'guest_age1', 'guest_hometown1', 
    'guest_name2','guest_job2', 'guest_age2', 'guest_hometown2', 'guest_name3',
    'guest_job3', 'guest_age3', 'guest_hometown3']]

,guest_name,guest_name1,guest_job1,guest_age1,guest_hometown1,guest_name2,guest_job2,guest_age2,guest_hometown2,guest_name3,guest_job3,guest_age3,guest_hometown3
1,"CHOON6, MANGO FRESH, DJ PANDA",CHOON6,NaN,NaN,NaN,MANGO FRESH,NaN,NaN,NaN,DJ PANDA,NaN,NaN,NaN
2,맹진환,맹진환,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"개코, 던밀스, 한해",개코,"래퍼, 작사가, 작곡가, 프로듀서",43,대한민국 서울특별시 강남구 신사동,던밀스,래퍼,36,대한민국 경기도 수원시,한해,"래퍼. 작사가, 작곡가, 프로듀서",34,대한민국 부산광역시
4,한해,한해,"래퍼. 작사가, 작곡가, 프로듀서",34,대한민국 부산광역시,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"유세윤, 뮤지",유세윤,정보 없음,44,대한민국 서울특별시 노원구 상계동[1],뮤지,None,None,None,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,양동근,양동근,"배우, 래퍼",45,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,"SOLE, 엄지윤",SOLE,가수,30,대한민국 부산광역시,엄지윤,코미디언,28,대한민국 서울특별시,NaN,NaN,NaN,NaN
105,이성우,이성우,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,한해,한해,"래퍼. 작사가, 작곡가, 프로듀서",34,대한민국 부산광역시,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [243]:
# # 엑셀로 내보내기

# with pd.ExcelWriter(r'E:\project\youtube\data\ZB_ML_성시경&최자로드_ver1.xlsx') as writer:
#     sung_updated_df.to_excel(writer, sheet_name='먹을텐데')
#     choi_updated_df.to_excel(writer, sheet_name='최자로드')